In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Step 1: Import Libraries

In [2]:
# Import Libraries

import time
import pickle
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from sklearn.svm import SVC
from sklearn.metrics import auc
from collections import Counter
warnings.simplefilter("ignore")
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from sklearn import preprocessing
from prettytable import PrettyTable 
from sklearn.metrics import roc_curve
from astropy.table import Table, Column
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, cohen_kappa_score, auc

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

###### SMOTE and RandomUnderSampler (for balancing data) ############

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

from imblearn.over_sampling import SVMSMOTE 
from imblearn.over_sampling import ADASYN 
from imblearn.under_sampling import NearMiss 

########### unsupervised learning for outliers ########################
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.cluster import DBSCAN


# Step 2: Read Dataset

In [3]:
# Load Dataset

data = pd.read_excel('drive/My Drive/Colab Notebooks/WisconsinDataset/Forms_cleaned.xlsx', 'form1')

print('\n\nDataset:')
print('========\n')
data



Dataset:



,EID,FormID,Flagged,Attempt,tot_time,iresp.1,iresp.2,iresp.3,iresp.4,iresp.5,...,idur.161,idur.162,idur.163,idur.164,idur.165,idur.166,idur.167,idur.168,idur.169,idur.170
0,e100001,form1,0,5,10133,1,1,3.0,1,2,...,17,102,55,50,17,28,53,82,100,21
1,e100002,form1,1,1,12409,1,3,3.0,1,1,...,22,121,114,44,36,53,120,93,87,29
2,e100003,form1,1,1,12457,2,3,4.0,4,2,...,73,34,92,83,50,30,63,96,79,48
3,e100004,form1,1,3,11389,1,1,1.0,4,2,...,30,93,49,26,57,41,59,41,84,17
4,e100005,form1,1,5,13669,1,1,2.0,4,2,...,11,0,58,127,63,68,164,74,63,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,e101632,form1,0,1,8760,1,1,3.0,3,2,...,10,72,40,10,10,52,56,47,61,19
1632,e101633,form1,0,1,10568,1,1,3.0,3,2,...,11,116,45,11,7,116,79,59,48,40
1633,e101634,form1,0,1,12449,1,1,3.0,3,2,...,47,164,28,11,30,54,19,27,43,14
1634,e101635,form1,0,1,8209,1,1,3.0,3,2,...,14,45,32,14,28,17,18,109,94,14


# Step 3: Understand Datset

In [4]:
# Understand Dataset

print("\n\nAttributes of Dataset:")
print("======================\n")

print("\nName of Dataset: Forms_cleaned")
print("==============================\n")

print("\nNumber of Rows:", len(data))
print("====================\n")

print("\nNumber of Columns:", len(data.columns))
print("======================\n")

print("\nClass Distribution:")
print("===================\n")
print(data['Flagged'].value_counts())



Attributes of Dataset:


Name of Dataset: Forms_cleaned


Number of Rows: 1636


Number of Columns: 515


Class Distribution:

0    1590
1      46
Name: Flagged, dtype: int64


# Step 4: Pre-Process Dataset

In [5]:
# Pre-Processing

data.tail()
data.drop(data.loc[:, 'iresp.1':'iresp.170'], inplace = True, axis = 1) 
data_final = data

In [6]:
# Count Null Values

print(data_final.isnull().sum())
print(data_final.shape)

EID         0
FormID      0
Flagged     0
Attempt     0
tot_time    0
           ..
idur.166    0
idur.167    0
idur.168    0
idur.169    0
idur.170    0
Length: 345, dtype: int64
(1636, 345)


In [7]:
# Time

data_final['mean_idur'] = data_final.loc[:, 'idur.1':'idur.170'].mean(axis=1)
data_final['median_idur'] = data_final.loc[:, 'idur.1':'idur.170'].median(axis=1)
data_final['max_idur'] = data_final.loc[:, 'idur.1':'idur.170'].max(axis=1)
data_final['min_idur'] = data_final.loc[:, 'idur.1':'idur.170'].min(axis=1)

# Correct Answers

data_final['iraw_score'] = data_final.loc[:, 'iraw.1':'iraw.170'].sum(axis=1)
data_final['iraw_score_pct'] = data_final['iraw_score']/170
data_final

,EID,FormID,Flagged,Attempt,tot_time,iraw.1,iraw.2,iraw.3,iraw.4,iraw.5,...,idur.167,idur.168,idur.169,idur.170,mean_idur,median_idur,max_idur,min_idur,iraw_score,iraw_score_pct
0,e100001,form1,0,5,10133,1,1,1,0,1,...,53,82,100,21,56.323529,50.0,143,14,54,0.317647
1,e100002,form1,1,1,12409,1,0,1,0,0,...,120,93,87,29,69.158824,63.0,202,11,55,0.323529
2,e100003,form1,1,1,12457,0,0,0,0,1,...,63,96,79,48,69.547059,65.5,179,16,61,0.358824
3,e100004,form1,1,3,11389,1,1,0,0,1,...,59,41,84,17,62.882353,62.0,154,12,61,0.358824
4,e100005,form1,1,5,13669,1,1,0,0,1,...,164,74,63,17,76.547059,69.0,189,0,62,0.364706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,e101632,form1,0,1,8760,1,1,1,1,1,...,56,47,61,19,48.658824,43.0,152,8,156,0.917647
1632,e101633,form1,0,1,10568,1,1,1,1,1,...,79,59,48,40,59.335294,49.0,230,7,153,0.900000
1633,e101634,form1,0,1,12449,1,1,1,1,1,...,19,27,43,14,69.894118,47.0,322,7,154,0.905882
1634,e101635,form1,0,1,8209,1,1,1,1,1,...,18,109,94,14,45.376471,34.0,258,10,152,0.894118


# Step 5: Feature Selection

In [8]:
# iraw_idur_OEF (both+OEF)

col_features = ['Attempt', 'tot_time', 'mean_idur', 'median_idur', 'max_idur', 'min_idur', 'iraw_score_pct']

for x in range(1,171):
    col_features.append('idur.'+str(x))

for i in range(1,171):
    col_features.append('iraw.'+str(i))

X = data_final.loc[:, col_features]
Y = data_final.Flagged    

In [9]:
# Count Values

counter = Counter(Y)
print(counter)

Counter({0: 1590, 1: 46})


# Step 7: Feature Scaling

In [10]:
# Import Libraries
from sklearn.preprocessing import MinMaxScaler
num_cols = X.columns

# apply standardization on numerical features
for i in num_cols:
    # fit on training data column
    scale = MinMaxScaler(feature_range=(0, 1)).fit(X[[i]])

    # transform the training data column
    X[i] = scale.transform(X[[i]])
    
print("\n\nTraining Features After Scaling: ")
print("=================================\n")
X



Training Features After Scaling: 



,Attempt,tot_time,mean_idur,median_idur,max_idur,min_idur,iraw_score_pct,idur.1,idur.2,idur.3,...,iraw.161,iraw.162,iraw.163,iraw.164,iraw.165,iraw.166,iraw.167,iraw.168,iraw.169,iraw.170
0,1.0,0.251140,0.254294,0.246154,0.037668,0.482759,0.000000,0.169065,0.041812,0.114007,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.2,0.373250,0.379208,0.379487,0.090583,0.379310,0.009804,0.151079,0.081882,0.250814,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.2,0.375825,0.382986,0.405128,0.069955,0.551724,0.068627,0.201439,0.055749,0.179153,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.6,0.318526,0.318125,0.369231,0.047534,0.413793,0.068627,0.219424,0.247387,0.153094,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0
4,1.0,0.440850,0.451111,0.441026,0.078924,0.000000,0.078431,0.406475,0.139373,0.250814,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,0.2,0.177477,0.179700,0.174359,0.045740,0.275862,1.000000,0.118705,0.073171,0.120521,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1632,0.2,0.274478,0.283604,0.235897,0.115695,0.241379,0.970588,0.064748,0.170732,0.127036,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1633,0.2,0.375396,0.386364,0.215385,0.198206,0.241379,0.980392,0.107914,0.134146,0.293160,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1634,0.2,0.147916,0.147756,0.082051,0.140807,0.344828,0.960784,0.025180,0.151568,0.166124,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0


# Step 8: Automatic Features Selection

<div class="alert alert-info">
<b>4 Automatic Features Selection Methods:</b>
<div class="alert alert-success">
<b>
    <code>(1)</code> Chi2 Features Selection <br><br>
    <code>(2)</code> ExtraTree Features Selection <br><br>
    <code>(3)</code> REF Features Selection <br><br>
    <code>(4)</code> Pearson Correlation Features Selection <br><br>
</b>
</div>
</div>

## Step 8.1: Chi2 Features Selection

## Step 8.2: Extra Tree Features Selection

<div class="alert alert-warning">
<b>This class implements a meta estimator that fits a number of randomized decision trees (a.k.a. extra-trees) on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.</b>
</div>

In [11]:
# Import Libraries
# from sklearn.ensemble import ExtraTreesClassifier

# model = ExtraTreesClassifier()
# model.fit(X,Y)
# columns = X.columns.values

# print(model.feature_importances_)
# feat_importances = pd.Series(model.feature_importances_)

In [12]:
# Print 20 Best Features

# top = feat_importances.nlargest(20)
# feature = top.index
# names = columns[feature]

# top_features = pd.DataFrame()
# top_features["Features"]=names
# top_features["    Score"]=top.values
# top_features["Ranking"] = ["       --->      Rank-01","       --->      Rank-02","       --->      Rank-03","       --->      Rank-04","       --->      Rank-05","       --->      Rank-06","       --->      Rank-07","       --->      Rank-08","       --->      Rank-09","       --->      Rank-10","       --->      Rank-11","       --->      Rank-12","       --->      Rank-13","       --->      Rank-14","       --->      Rank-15","       --->      Rank-16","       --->      Rank-17","       --->      Rank-18","       --->      Rank-19","       --->      Rank-20"]
# print("\n===================================================\n")
# print("================== TOP 10 FEATURES ================")
# print("\n===================================================\n")
# print(top_features.to_string(index=False))
# print("\n===================================================\n")

### Features Set-2

In [13]:
# Features_Set2 = X[top_features["Features"]]

# print("\n\nAutomatic Selected Features Set 2:")
# print("==================================\n")
# Features_Set2

<center>
    <br>
        <br>
            <br>
                <h5 style = "color:blue">====================================================================</h5>
                <h2 style = "color:green">Training Phase</h2>
                <h5 style = "color:blue">====================================================================</h5>
            <br>
        <br>
    <br>
</center>

## Step 12.1: Train Test Split

In [14]:
# Train Test Split

#X_train, X_test, y_train, y_test= train_test_split(Features_Set2, Y, test_size= 0.2, random_state=64 ,shuffle = True)

##### top 20 features by 1 selection alg. #########
# X_train, X_test, y_train, y_test= train_test_split(Features_Set2, Y, test_size= 0.2, random_state=23 ,shuffle = True)
# X_train1, X_pred_train, y_train1, y_pred_train= train_test_split(X_train, y_train, test_size= 0.25, random_state=23 ,shuffle = True)

##### X, Y ##########
# X_train, X_test, y_train, y_test= train_test_split(X, Y, test_size= 0.2, random_state=23 ,shuffle = True)
# X_train1, X_pred_train, y_train1, y_pred_train= train_test_split(X_train, y_train, test_size= 0.25, random_state=23 ,shuffle = True)


##### top features from 4 selection alg. plus summatives, and resampling ##########
### both 33%
over = SMOTE(sampling_strategy=0.4)
#over = SVMSMOTE(sampling_strategy=0.4, random_state=42, k_neighbors=10)
#over = ADASYN(sampling_strategy=0.4, random_state=42)  ###bad

under = RandomUnderSampler(sampling_strategy=0.5)
#under = NearMiss(sampling_strategy=0.5)

steps = [('o', over), ('u', under)]

pipeline = Pipeline(steps=steps)

X_train, X_test, y_train, y_test= train_test_split(X, Y, test_size= 0.2, random_state=23 ,shuffle = True)
X_train1_orig, X_pred_train, y_train1_orig, y_pred_train= train_test_split(X_train, y_train, test_size= 0.25, random_state=23 ,shuffle = True)

X_train1, y_train1 = pipeline.fit_resample(X_train1_orig, y_train1_orig)


## Step 12.3: Random Forest Classifier

In [15]:
# Training

clf1 = RandomForestClassifier(criterion="gini", max_depth=500, max_features="auto", min_samples_leaf=1, min_samples_split=2, n_estimators=911)
clf1.fit(X_train1, y_train1)

RandomForestClassifier(max_depth=500, n_estimators=911)

### Evalute the performace

In [16]:
# Predictions
y_pred = clf1.predict(X_pred_train)


tn, fp, fn, tp = confusion_matrix(y_pred_train, y_pred).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  precision: ", precision_score(y_pred_train, y_pred))
print("  recall: ", recall_score(y_pred_train, y_pred))
print("  f1-score: ", f1_score(y_pred_train, y_pred))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

  precision:  0.0
  recall:  0.0
  f1-score:  0.0
  fpr:  0.0
  tn:  320
  fp:  0
  fn:  7
  tp:  0
  real_cheaters:  7


## Step 12.6: Gradient Boosting Classifier

In [17]:
# Training

clf2 = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100)
clf2.fit(X_train1, y_train1)


GradientBoostingClassifier()

### Evalute the performace

In [18]:
# Predictions
y_pred = clf2.predict(X_pred_train)

tn, fp, fn, tp = confusion_matrix(y_pred_train, y_pred).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  precision: ", precision_score(y_pred_train, y_pred))
print("  recall: ", recall_score(y_pred_train, y_pred))
print("  f1-score: ", f1_score(y_pred_train, y_pred))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

  precision:  0.0
  recall:  0.0
  f1-score:  0.0
  fpr:  0.0
  tn:  320
  fp:  0
  fn:  7
  tp:  0
  real_cheaters:  7


In [19]:
clf3 = MLPClassifier(activation="relu", learning_rate="invscaling", solver="adam")
clf3.fit(X_train1, y_train1)

# Predictions
y_pred = clf3.predict(X_pred_train)

tn, fp, fn, tp = confusion_matrix(y_pred_train, y_pred).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  precision: ", precision_score(y_pred_train, y_pred))
print("  recall: ", recall_score(y_pred_train, y_pred))
print("  f1-score: ", f1_score(y_pred_train, y_pred))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

  precision:  0.2857142857142857
  recall:  0.2857142857142857
  f1-score:  0.2857142857142857
  fpr:  0.015625
  tn:  315
  fp:  5
  fn:  5
  tp:  2
  real_cheaters:  7


In [20]:
clf4 = GaussianNB(var_smoothing=6.579332246575682e-08)
clf4.fit(X_train1, y_train1)

# Predictions
y_pred = clf4.predict(X_pred_train)

tn, fp, fn, tp = confusion_matrix(y_pred_train, y_pred).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  precision: ", precision_score(y_pred_train, y_pred))
print("  recall: ", recall_score(y_pred_train, y_pred))
print("  f1-score: ", f1_score(y_pred_train, y_pred))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

  precision:  0.05405405405405406
  recall:  0.2857142857142857
  f1-score:  0.09090909090909091
  fpr:  0.109375
  tn:  285
  fp:  35
  fn:  5
  tp:  2
  real_cheaters:  7


In [21]:
clf5 = LinearDiscriminantAnalysis(shrinkage="auto", solver="eigen")
clf5.fit(X_train1, y_train1)

clf6 = LogisticRegression(C=2, multi_class="multinomial", solver="newton-cg")
clf6.fit(X_train1, y_train1)

clf7 = DecisionTreeClassifier(criterion="entropy", max_features="auto", splitter="best")
clf7.fit(X_train1, y_train1)

clf8 = SVC(C=2, gamma="auto", kernel="rbf", probability = True)
clf8.fit(X_train1, y_train1)


SVC(C=2, gamma='auto', probability=True)

## constructing Meta model  

In [22]:
############ 5 outliers on pred_train set ##############

Isolation_Forest = IsolationForest(random_state=64)
outlier1 = Isolation_Forest.fit_predict(X_pred_train)
print("Number Outliers Detected: ", list(outlier1).count(-1))
print("Other 1: ", list(outlier1).count(1))

Elliptic_Envelope = EllipticEnvelope()
outlier2 = Elliptic_Envelope.fit_predict(X_pred_train)
print("Number Outliers Detected: ", list(outlier2).count(-1))
print("Other 1: ", list(outlier2).count(1))

Local_Outlier_Factor = LocalOutlierFactor()
outlier3 = Local_Outlier_Factor.fit_predict(X_pred_train)
print("Number Outliers Detected: ", list(outlier3).count(-1))
print("Other 1: ", list(outlier3).count(1))

One_Class_SVM = OneClassSVM(kernel='linear', tol=0.001, nu=0.05)
outlier4 = One_Class_SVM.fit_predict(X_pred_train)
print("Number Outliers Detected: ", list(outlier4).count(-1))
print("Other 1: ", list(outlier4).count(1))

outlier_detection = DBSCAN(min_samples = 300, eps = 8.2)
outlier5 = outlier_detection.fit_predict(X_pred_train)
print("Number Outliers Detected: ", list(outlier5).count(-1))
print("Other 0: ", list(outlier5).count(0))


Number Outliers Detected:  36
Other 1:  291
Number Outliers Detected:  27
Other 1:  300
Number Outliers Detected:  0
Other 1:  327
Number Outliers Detected:  16
Other 1:  311
Number Outliers Detected:  22
Other 0:  305


In [23]:
############  predictions on pre_train set ##############

P1 = clf1.predict_proba(X_pred_train)
P1 = P1[:, 1]
P2 = clf2.predict_proba(X_pred_train)
P2 = P2[:, 1]
P3 = clf3.predict_proba(X_pred_train)
P3 = P3[:, 1]
P4 = clf4.predict_proba(X_pred_train)
P4 = P4[:, 1]

P5 = clf5.predict_proba(X_pred_train)
P5 = P5[:, 1]
P6 = clf6.predict_proba(X_pred_train)
P6 = P6[:, 1]
P7 = clf7.predict_proba(X_pred_train)
P7 = P7[:, 1]
P8 = clf8.predict_proba(X_pred_train)
P8 = P8[:, 1]

In [24]:

X_pred_train["Predict1"] = P1
X_pred_train["Predict2"] = P2
X_pred_train["Predict3"] = P3
X_pred_train["Predict4"] = P4

X_pred_train["Predict5"] = P5
X_pred_train["Predict6"] = P6
X_pred_train["Predict7"] = P7
X_pred_train["Predict8"] = P8

X_pred_train["Outlier1"] = outlier1
X_pred_train["Outlier2"] = outlier2
X_pred_train["Outlier3"] = outlier3
X_pred_train["Outlier4"] = outlier4
X_pred_train["Outlier5"] = outlier5

X_pred_train

,Attempt,tot_time,mean_idur,median_idur,max_idur,min_idur,iraw_score_pct,idur.1,idur.2,idur.3,...,Predict4,Predict5,Predict6,Predict7,Predict8,Outlier1,Outlier2,Outlier3,Outlier4,Outlier5
1204,0.8,0.211546,0.221204,0.189744,0.066368,0.379310,0.656863,0.187050,0.090592,0.100977,...,7.584631e-43,1.053921e-07,0.000049,0.0,0.000531,1,1,1,1,0
1222,0.6,0.385858,0.381383,0.328205,0.095964,0.413793,0.686275,0.194245,0.043554,0.114007,...,9.544686e-51,5.040032e-10,0.000159,0.0,0.000233,1,-1,1,1,0
653,0.2,0.369548,0.359171,0.297436,0.200897,0.275862,0.843137,0.032374,0.026132,0.091205,...,5.369670e-105,8.598635e-10,0.000446,0.0,0.000573,1,1,1,1,0
1482,0.2,0.147272,0.145638,0.158974,0.019731,0.275862,0.794118,0.089928,0.121951,0.087948,...,1.000000e+00,5.707290e-07,0.001975,0.0,0.013168,1,1,1,1,0
377,0.4,0.384409,0.370678,0.328205,0.327354,0.413793,0.656863,0.323741,0.247387,0.355049,...,8.825601e-55,8.425915e-07,0.007686,0.0,0.011765,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1626,0.2,0.378937,0.385562,0.384615,0.086996,0.551724,0.950980,0.258993,0.099303,0.061889,...,1.626021e-33,5.554239e-08,0.000320,0.0,0.000272,1,1,1,1,0
462,0.2,0.396588,0.405370,0.317949,0.117489,0.448276,0.705882,0.327338,0.236934,0.084691,...,1.062549e-62,3.734030e-12,0.000010,0.0,0.000035,1,1,1,1,0
1374,0.2,0.346531,0.358370,0.256410,0.124664,0.310345,0.794118,0.287770,0.216028,0.110749,...,2.735879e-100,3.826704e-12,0.000660,0.0,0.001137,1,1,1,1,0
1574,0.2,0.207415,0.210671,0.169231,0.081614,0.241379,0.843137,0.115108,0.275261,0.048860,...,3.496235e-28,3.880303e-02,0.018861,0.0,0.007324,1,-1,1,1,0


In [25]:

# summatives + top features from 4 selection algorithm + prediction1-4
col_features = ['Attempt', 'tot_time', 'mean_idur', 'median_idur', 'max_idur', 'min_idur', 'iraw_score_pct', 'iraw.32', 'iraw.77', 'iraw.147', 'iraw.57', 'iraw.86', 'iraw.152', 'iraw.12', 'iraw.127', 'iraw.140', 'iraw.96', 'idur.146', 'idur.36', 'idur.114', 'idur.84', 'idur.142', 'idur.60', 'idur.71', 'idur.133', 'idur.88', 'idur.59', 'idur.69', 'idur.38', 'idur.67', 'idur.81', 'idur.96', 'idur.162', 'Predict1', 'Predict2', 'Outlier1', 'Outlier2', 'Outlier3', 'Outlier4', 'Outlier5']
#col_features = ['Predict1', 'Predict2', 'Predict3', 'Predict4', 'Predict5', 'Outlier1', 'Outlier2', 'Outlier3', 'Outlier4', 'Outlier5']
#col_features = ['Predict1', 'Predict2']

X_pred_train_meta = X_pred_train.loc[:, col_features]


In [26]:
X_pred_train_meta

,Attempt,tot_time,mean_idur,median_idur,max_idur,min_idur,iraw_score_pct,iraw.32,iraw.77,iraw.147,...,idur.81,idur.96,idur.162,Predict1,Predict2,Outlier1,Outlier2,Outlier3,Outlier4,Outlier5
1204,0.8,0.211546,0.221204,0.189744,0.066368,0.379310,0.656863,1.0,1.0,0.0,...,0.102094,0.120,0.182561,0.041712,0.001309,1,1,1,1,0
1222,0.6,0.385858,0.381383,0.328205,0.095964,0.413793,0.686275,0.0,0.0,0.0,...,0.465969,0.740,0.332425,0.063666,0.000677,1,-1,1,1,0
653,0.2,0.369548,0.359171,0.297436,0.200897,0.275862,0.843137,1.0,0.0,0.0,...,0.065445,0.325,0.234332,0.030735,0.000687,1,1,1,1,0
1482,0.2,0.147272,0.145638,0.158974,0.019731,0.275862,0.794118,1.0,0.0,0.0,...,0.167539,0.245,0.152589,0.104281,0.000910,1,1,1,1,0
377,0.4,0.384409,0.370678,0.328205,0.327354,0.413793,0.656863,0.0,0.0,0.0,...,0.209424,0.175,0.152589,0.119649,0.010498,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1626,0.2,0.378937,0.385562,0.384615,0.086996,0.551724,0.950980,1.0,0.0,0.0,...,0.416230,0.310,0.305177,0.009879,0.000345,1,1,1,1,0
462,0.2,0.396588,0.405370,0.317949,0.117489,0.448276,0.705882,1.0,1.0,1.0,...,0.073298,0.285,0.193460,0.020856,0.000211,1,1,1,1,0
1374,0.2,0.346531,0.358370,0.256410,0.124664,0.310345,0.794118,1.0,1.0,0.0,...,0.170157,0.100,0.629428,0.039517,0.000765,1,1,1,1,0
1574,0.2,0.207415,0.210671,0.169231,0.081614,0.241379,0.843137,1.0,0.0,0.0,...,0.060209,0.250,0.160763,0.072448,0.001484,1,-1,1,1,0


In [27]:
combined_model_gb = GradientBoostingClassifier()  
combined_model_rf = RandomForestClassifier()
combined_model_nb = GaussianNB()  
combined_model_lr = LogisticRegression()
combined_model_svm = SVC(kernel='linear', probability=True)
combined_model_dt = DecisionTreeClassifier()
combined_model_lda = LinearDiscriminantAnalysis(shrinkage="auto", solver="eigen") 
combined_model_nn = MLPClassifier(activation="relu", learning_rate="invscaling", solver="adam")


combined_model_gb.fit(X_pred_train_meta, y_pred_train)
combined_model_rf.fit(X_pred_train_meta, y_pred_train)
combined_model_nb.fit(X_pred_train_meta, y_pred_train)
combined_model_lr.fit(X_pred_train_meta, y_pred_train)
combined_model_svm.fit(X_pred_train_meta, y_pred_train)
combined_model_dt.fit(X_pred_train_meta, y_pred_train)
combined_model_lda.fit(X_pred_train_meta, y_pred_train)
combined_model_nn.fit(X_pred_train_meta, y_pred_train)


MLPClassifier(learning_rate='invscaling')

## Test set

In [28]:
import xlwt 
from xlwt import Workbook 
  
wb = Workbook() 

sheet = wb.add_sheet('sheet1') 

sheet.write(0, 0, 'Model')
sheet.write(0, 1, 'Precision')
sheet.write(0, 2, 'Recall')
sheet.write(0, 3, 'F1 Score')
sheet.write(0, 4, 'weighted Kappa')
sheet.write(0, 5, 'ROC AUC')
sheet.write(0, 6, 'FPR')
sheet.write(0, 7, 'meta')

In [29]:
# scores on test data
###
probs = clf1.predict_proba(X_test)
probs = probs[:, 1]
y_pred = (probs >= 0.5).astype(bool)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  -------------------- RF ------------------------- ")
print("  precision: ", precision_score(y_test, y_pred))
print("  recall: ", recall_score(y_test, y_pred))
print("  f1-score: ", f1_score(y_test, y_pred))
print("  kappa: ", cohen_kappa_score(y_test, y_pred, weights = 'quadratic'))
print("  roc: ", roc_auc_score(y_test, probs))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

# sheet.write(1, 0, "Random Forest")
# sheet.write(1, 1, precision_score(y_test, y_pred))
# sheet.write(1, 2, recall_score(y_test, y_pred))
# sheet.write(1, 3, f1_score(y_test, y_pred))
# sheet.write(1, 4, cohen_kappa_score(y_test, y_pred, weights = 'quadratic'))
# sheet.write(1, 5, roc_auc_score(y_test, probs))
# sheet.write(1, 6, fpr)

###
probs = clf2.predict_proba(X_test)
probs = probs[:, 1]
y_pred = (probs >= 0.5).astype(bool)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  -------------------- GB ------------------------- ")
print("  precision: ", precision_score(y_test, y_pred))
print("  recall: ", recall_score(y_test, y_pred))
print("  f1-score: ", f1_score(y_test, y_pred))
print("  kappa: ", cohen_kappa_score(y_test, y_pred, weights = 'quadratic'))
print("  roc: ", roc_auc_score(y_test, probs))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

# sheet.write(2, 0, "Gradient Boosting")
# sheet.write(2, 1, precision_score(y_test, y_pred))
# sheet.write(2, 2, recall_score(y_test, y_pred))
# sheet.write(2, 3, f1_score(y_test, y_pred))
# sheet.write(2, 4, cohen_kappa_score(y_test, y_pred, weights = 'quadratic'))
# sheet.write(2, 5, roc_auc_score(y_test, probs))
# sheet.write(2, 6, fpr)

###
probs = clf3.predict_proba(X_test)
probs = probs[:, 1]
y_pred = (probs >= 0.5).astype(bool)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  -------------------- NN ------------------------- ")
print("  precision: ", precision_score(y_test, y_pred))
print("  recall: ", recall_score(y_test, y_pred))
print("  f1-score: ", f1_score(y_test, y_pred))
print("  kappa: ", cohen_kappa_score(y_test, y_pred, weights = 'quadratic'))
print("  roc: ", roc_auc_score(y_test, probs))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

# sheet.write(3, 0, "Neural Network")
# sheet.write(3, 1, precision_score(y_test, y_pred))
# sheet.write(3, 2, recall_score(y_test, y_pred))
# sheet.write(3, 3, f1_score(y_test, y_pred))
# sheet.write(3, 4, cohen_kappa_score(y_test, y_pred, weights = 'quadratic'))
# sheet.write(3, 5, roc_auc_score(y_test, probs))
# sheet.write(3, 6, fpr)

###
probs = clf4.predict_proba(X_test)
probs = probs[:, 1]
y_pred = (probs >= 0.5).astype(bool)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  -------------------- NB ------------------------- ")
print("  precision: ", precision_score(y_test, y_pred))
print("  recall: ", recall_score(y_test, y_pred))
print("  f1-score: ", f1_score(y_test, y_pred))
print("  kappa: ", cohen_kappa_score(y_test, y_pred, weights = 'quadratic'))
print("  roc: ", roc_auc_score(y_test, probs))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

# sheet.write(4, 0, "Naive Bayes")
# sheet.write(4, 1, precision_score(y_test, y_pred))
# sheet.write(4, 2, recall_score(y_test, y_pred))
# sheet.write(4, 3, f1_score(y_test, y_pred))
# sheet.write(4, 4, cohen_kappa_score(y_test, y_pred, weights = 'quadratic'))
# sheet.write(4, 5, roc_auc_score(y_test, probs))
# sheet.write(4, 6, fpr)

  -------------------- RF ------------------------- 
  precision:  0.0
  recall:  0.0
  f1-score:  0.0
  kappa:  0.0
  roc:  0.9539936102236422
  fpr:  0.0
  tn:  313
  fp:  0
  fn:  15
  tp:  0
  real_cheaters:  15
  -------------------- GB ------------------------- 
  precision:  1.0
  recall:  0.13333333333333333
  f1-score:  0.23529411764705882
  kappa:  0.22697606961566352
  roc:  0.9243876464323748
  fpr:  0.0
  tn:  313
  fp:  0
  fn:  13
  tp:  2
  real_cheaters:  15
  -------------------- NN ------------------------- 
  precision:  0.75
  recall:  0.2
  f1-score:  0.31578947368421056
  kappa:  0.3023560209424083
  roc:  0.8984025559105431
  fpr:  0.003194888178913738
  tn:  312
  fp:  1
  fn:  12
  tp:  3
  real_cheaters:  15
  -------------------- NB ------------------------- 
  precision:  0.3
  recall:  0.4
  f1-score:  0.34285714285714286
  kappa:  0.30661764705882355
  roc:  0.7908413205537806
  fpr:  0.04472843450479233
  tn:  299
  fp:  14
  fn:  9
  tp:  6
  real_cheat

In [30]:
############ 5 outliers on test set ##############

Isolation_Forest = IsolationForest(random_state=64)
outlier1_test = Isolation_Forest.fit_predict(X_test)
print("Number Outliers Detected: ", list(outlier1_test).count(-1))
print("Other 1: ", list(outlier1_test).count(1))

Elliptic_Envelope = EllipticEnvelope()
outlier2_test = Elliptic_Envelope.fit_predict(X_test)
print("Number Outliers Detected: ", list(outlier2_test).count(-1))
print("Other 1: ", list(outlier2_test).count(1))

Local_Outlier_Factor = LocalOutlierFactor()
outlier3_test = Local_Outlier_Factor.fit_predict(X_test)
print("Number Outliers Detected: ", list(outlier3_test).count(-1))
print("Other 1: ", list(outlier3_test).count(1))

One_Class_SVM = OneClassSVM(kernel='linear', tol=0.001, nu=0.05)
outlier4_test = One_Class_SVM.fit_predict(X_test)
print("Number Outliers Detected: ", list(outlier4_test).count(-1))
print("Other 1: ", list(outlier4_test).count(1))

outlier_detection = DBSCAN(min_samples = 300, eps = 8.5)
outlier5_test = outlier_detection.fit_predict(X_test)
print("Number Outliers Detected: ", list(outlier5_test).count(-1))
print("Other 0: ", list(outlier5_test).count(0))


Number Outliers Detected:  36
Other 1:  292
Number Outliers Detected:  33
Other 1:  295
Number Outliers Detected:  0
Other 1:  328
Number Outliers Detected:  15
Other 1:  313
Number Outliers Detected:  20
Other 0:  308


In [31]:
############ predictions on test set ################

P1_test = clf1.predict_proba(X_test)
P1_test = P1_test[:, 1]
P2_test = clf2.predict_proba(X_test)
P2_test = P2_test[:, 1]
P3_test = clf3.predict_proba(X_test)
P3_test = P3_test[:, 1]
P4_test = clf4.predict_proba(X_test)
P4_test = P4_test[:, 1]

P5_test = clf5.predict_proba(X_test)
P5_test = P5_test[:, 1]
P6_test = clf6.predict_proba(X_test)
P6_test = P6_test[:, 1]
P7_test = clf7.predict_proba(X_test)
P7_test = P7_test[:, 1]
P8_test = clf8.predict_proba(X_test)
P8_test = P8_test[:, 1]

In [32]:
X_test["Predict1"] = P1_test
X_test["Predict2"] = P2_test
X_test["Predict3"] = P3_test
X_test["Predict4"] = P4_test
X_test["Predict5"] = P5_test
X_test["Predict6"] = P6_test
X_test["Predict7"] = P7_test
X_test["Predict8"] = P8_test

X_test["Outlier1"] = outlier1_test
X_test["Outlier2"] = outlier2_test
X_test["Outlier3"] = outlier3_test
X_test["Outlier4"] = outlier4_test
X_test["Outlier5"] = outlier5_test

X_test_meta = X_test.loc[:, col_features]   ### top features + prediction1~4 + outlier1~5

# combined_model_probs = combined_model.predict_proba(X_test_meta)
# combined_model_probs = combined_model_probs[:, 1]
# combined_model_predictions = (combined_model_probs >= 0.5).astype(bool)

# combined_model_predictions = combined_model.predict(X_test_meta)

In [33]:
combined_model_probs = combined_model_gb.predict_proba(X_test_meta)
combined_model_probs = combined_model_probs[:, 1]
combined_model_predictions = (combined_model_probs >= 0.5).astype(bool)

tn, fp, fn, tp = confusion_matrix(y_test, combined_model_predictions).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  precision: ", precision_score(y_test, combined_model_predictions))
print("  recall: ", recall_score(y_test, combined_model_predictions))
print("  f1-score: ", f1_score(y_test, combined_model_predictions))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

sheet.write(1, 0, "Stacking")
sheet.write(1, 1, precision_score(y_test, combined_model_predictions))
sheet.write(1, 2, recall_score(y_test, combined_model_predictions))
sheet.write(1, 3, f1_score(y_test, combined_model_predictions))
sheet.write(1, 4, cohen_kappa_score(y_test, combined_model_predictions, weights = 'quadratic'))
sheet.write(1, 5, roc_auc_score(y_test, combined_model_probs))
sheet.write(1, 6, fpr)
sheet.write(1, 7, "GB")


  precision:  0.8461538461538461
  recall:  0.7333333333333333
  f1-score:  0.7857142857142856
  fpr:  0.006389776357827476
  tn:  311
  fp:  2
  fn:  4
  tp:  11
  real_cheaters:  15


In [34]:
combined_model_probs = combined_model_rf.predict_proba(X_test_meta)
combined_model_probs = combined_model_probs[:, 1]
combined_model_predictions = (combined_model_probs >= 0.5).astype(bool)

tn, fp, fn, tp = confusion_matrix(y_test, combined_model_predictions).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  precision: ", precision_score(y_test, combined_model_predictions))
print("  recall: ", recall_score(y_test, combined_model_predictions))
print("  f1-score: ", f1_score(y_test, combined_model_predictions))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

sheet.write(2, 0, "Stacking")
sheet.write(2, 1, precision_score(y_test, combined_model_predictions))
sheet.write(2, 2, recall_score(y_test, combined_model_predictions))
sheet.write(2, 3, f1_score(y_test, combined_model_predictions))
sheet.write(2, 4, cohen_kappa_score(y_test, combined_model_predictions, weights = 'quadratic'))
sheet.write(2, 5, roc_auc_score(y_test, combined_model_probs))
sheet.write(2, 6, fpr)
sheet.write(2, 7, "RF")

  precision:  0.875
  recall:  0.4666666666666667
  f1-score:  0.608695652173913
  fpr:  0.003194888178913738
  tn:  312
  fp:  1
  fn:  8
  tp:  7
  real_cheaters:  15


In [35]:
combined_model_probs = combined_model_nb.predict_proba(X_test_meta)
combined_model_probs = combined_model_probs[:, 1]
combined_model_predictions = (combined_model_probs >= 0.5).astype(bool)

tn, fp, fn, tp = confusion_matrix(y_test, combined_model_predictions).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  precision: ", precision_score(y_test, combined_model_predictions))
print("  recall: ", recall_score(y_test, combined_model_predictions))
print("  f1-score: ", f1_score(y_test, combined_model_predictions))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

sheet.write(3, 0, "Stacking")
sheet.write(3, 1, precision_score(y_test, combined_model_predictions))
sheet.write(3, 2, recall_score(y_test, combined_model_predictions))
sheet.write(3, 3, f1_score(y_test, combined_model_predictions))
sheet.write(3, 4, cohen_kappa_score(y_test, combined_model_predictions, weights = 'quadratic'))
sheet.write(3, 5, roc_auc_score(y_test, combined_model_probs))
sheet.write(3, 6, fpr)
sheet.write(3, 7, "NB")

  precision:  0.051094890510948905
  recall:  0.4666666666666667
  f1-score:  0.09210526315789473
  fpr:  0.41533546325878595
  tn:  183
  fp:  130
  fn:  8
  tp:  7
  real_cheaters:  15


In [36]:
combined_model_probs = combined_model_lr.predict_proba(X_test_meta)
combined_model_probs = combined_model_probs[:, 1]
combined_model_predictions = (combined_model_probs >= 0.5).astype(bool)

tn, fp, fn, tp = confusion_matrix(y_test, combined_model_predictions).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  precision: ", precision_score(y_test, combined_model_predictions))
print("  recall: ", recall_score(y_test, combined_model_predictions))
print("  f1-score: ", f1_score(y_test, combined_model_predictions))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

sheet.write(4, 0, "Stacking")
sheet.write(4, 1, precision_score(y_test, combined_model_predictions))
sheet.write(4, 2, recall_score(y_test, combined_model_predictions))
sheet.write(4, 3, f1_score(y_test, combined_model_predictions))
sheet.write(4, 4, cohen_kappa_score(y_test, combined_model_predictions, weights = 'quadratic'))
sheet.write(4, 5, roc_auc_score(y_test, combined_model_probs))
sheet.write(4, 6, fpr)
sheet.write(4, 7, "LR")

  precision:  0.0
  recall:  0.0
  f1-score:  0.0
  fpr:  0.0
  tn:  313
  fp:  0
  fn:  15
  tp:  0
  real_cheaters:  15


In [37]:
combined_model_probs = combined_model_svm.predict_proba(X_test_meta)
combined_model_probs = combined_model_probs[:, 1]
combined_model_predictions = (combined_model_probs >= 0.5).astype(bool)

tn, fp, fn, tp = confusion_matrix(y_test, combined_model_predictions).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  precision: ", precision_score(y_test, combined_model_predictions))
print("  recall: ", recall_score(y_test, combined_model_predictions))
print("  f1-score: ", f1_score(y_test, combined_model_predictions))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

sheet.write(5, 0, "Stacking")
sheet.write(5, 1, precision_score(y_test, combined_model_predictions))
sheet.write(5, 2, recall_score(y_test, combined_model_predictions))
sheet.write(5, 3, f1_score(y_test, combined_model_predictions))
sheet.write(5, 4, cohen_kappa_score(y_test, combined_model_predictions, weights = 'quadratic'))
sheet.write(5, 5, roc_auc_score(y_test, combined_model_probs))
sheet.write(5, 6, fpr)
sheet.write(5, 7, "SVM")

  precision:  0.0
  recall:  0.0
  f1-score:  0.0
  fpr:  0.0
  tn:  313
  fp:  0
  fn:  15
  tp:  0
  real_cheaters:  15


In [38]:
combined_model_probs = combined_model_dt.predict_proba(X_test_meta)
combined_model_probs = combined_model_probs[:, 1]
combined_model_predictions = (combined_model_probs >= 0.5).astype(bool)

tn, fp, fn, tp = confusion_matrix(y_test, combined_model_predictions).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  precision: ", precision_score(y_test, combined_model_predictions))
print("  recall: ", recall_score(y_test, combined_model_predictions))
print("  f1-score: ", f1_score(y_test, combined_model_predictions))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

sheet.write(6, 0, "Stacking")
sheet.write(6, 1, precision_score(y_test, combined_model_predictions))
sheet.write(6, 2, recall_score(y_test, combined_model_predictions))
sheet.write(6, 3, f1_score(y_test, combined_model_predictions))
sheet.write(6, 4, cohen_kappa_score(y_test, combined_model_predictions, weights = 'quadratic'))
sheet.write(6, 5, roc_auc_score(y_test, combined_model_probs))
sheet.write(6, 6, fpr)
sheet.write(6, 7, "DT")

  precision:  0.6875
  recall:  0.7333333333333333
  f1-score:  0.7096774193548386
  fpr:  0.01597444089456869
  tn:  308
  fp:  5
  fn:  4
  tp:  11
  real_cheaters:  15


In [39]:
combined_model_probs = combined_model_lda.predict_proba(X_test_meta)
combined_model_probs = combined_model_probs[:, 1]
combined_model_predictions = (combined_model_probs >= 0.5).astype(bool)

tn, fp, fn, tp = confusion_matrix(y_test, combined_model_predictions).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  precision: ", precision_score(y_test, combined_model_predictions))
print("  recall: ", recall_score(y_test, combined_model_predictions))
print("  f1-score: ", f1_score(y_test, combined_model_predictions))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

sheet.write(7, 0, "Stacking")
sheet.write(7, 1, precision_score(y_test, combined_model_predictions))
sheet.write(7, 2, recall_score(y_test, combined_model_predictions))
sheet.write(7, 3, f1_score(y_test, combined_model_predictions))
sheet.write(7, 4, cohen_kappa_score(y_test, combined_model_predictions, weights = 'quadratic'))
sheet.write(7, 5, roc_auc_score(y_test, combined_model_probs))
sheet.write(7, 6, fpr)
sheet.write(7, 7, "DA")

  precision:  0.7692307692307693
  recall:  0.6666666666666666
  f1-score:  0.7142857142857142
  fpr:  0.009584664536741214
  tn:  310
  fp:  3
  fn:  5
  tp:  10
  real_cheaters:  15


In [40]:
combined_model_probs = combined_model_nn.predict_proba(X_test_meta)
combined_model_probs = combined_model_probs[:, 1]
combined_model_predictions = (combined_model_probs >= 0.5).astype(bool)

tn, fp, fn, tp = confusion_matrix(y_test, combined_model_predictions).ravel()
fpr = (fp)/(fp+tn)
real_cheater = tp+fn

print("  precision: ", precision_score(y_test, combined_model_predictions))
print("  recall: ", recall_score(y_test, combined_model_predictions))
print("  f1-score: ", f1_score(y_test, combined_model_predictions))

print("  fpr: ", fpr)
print("  tn: ", tn)
print("  fp: ", fp)
print("  fn: ", fn)
print("  tp: ", tp)
print("  real_cheaters: ", real_cheater)

sheet.write(8, 0, "Stacking")
sheet.write(8, 1, precision_score(y_test, combined_model_predictions))
sheet.write(8, 2, recall_score(y_test, combined_model_predictions))
sheet.write(8, 3, f1_score(y_test, combined_model_predictions))
sheet.write(8, 4, cohen_kappa_score(y_test, combined_model_predictions, weights = 'quadratic'))
sheet.write(8, 5, roc_auc_score(y_test, combined_model_probs))
sheet.write(8, 6, fpr)
sheet.write(8, 7, "NN")

  precision:  0.5
  recall:  0.06666666666666667
  f1-score:  0.11764705882352941
  fpr:  0.003194888178913738
  tn:  312
  fp:  1
  fn:  14
  tp:  1
  real_cheaters:  15


In [41]:
wb.save('stacking2_outlier_topfeature.xls') 